# <center/>将数据集转换为MindSpore数据格式

## 概述

用户可以将非标准的数据集转换为MindSpore数据格式—MindRecord，从而方便地加载到MindSpore中进行训练。同时，MindSpore在部分场景做了性能优化，使用MindRecord数据格式可以获得更好的性能体验。

MindSpore数据格式具备的特征如下：
- 实现多变的用户数据统一存储、访问，训练数据读取更加简便。
- 数据聚合存储，高效读取，且方便管理、移动。
- 高效的数据编解码操作，对用户透明、无感知。
- 可以灵活控制分区的大小，实现分布式训练。

## 整体流程

1. 准备环节。
2. 将MNIST数据集转换为MindSpore数据格式。
3. 将CSV数据集转换为MindSpore数据格式。
4. 将CIFAR-10数据集转换为MindSpore数据格式。
5. 将CIFAR-100数据集转换为MindSpore数据格式。
6. 将ImageNet数据集转换为MindSpore数据格式。
7. 用户自定义生成MindSpore数据格式。

## 准备环节

### 导入模块

该模块提供API以加载和处理数据集。

In [1]:
import mindspore.dataset as ds

### 创建目录

- 在jupyter工作目录下创建dataset目录，本次体验将所有的原始数据集都放在该目录下。
- 在jupyter工作目录下创建transform目录，本次体验将所有的转换数据集都放在该目录下。

## 将MNIST数据集转换为MindSpore数据格式

### MNIST数据集下载

- 训练数据集：
   > http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz
   > http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz
- 测试数据集：
   > http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz
   > http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz
- 将下载好的数据集放在`jupyter工作目录/dataset/MnistData/`下，如图所示：

    dataset/MnistData/  
    ├── t10k-images-idx3-ubyte.gz  
    ├── t10k-labels-idx1-ubyte.gz  
    ├── train-images-idx3-ubyte.gz  
    └── train-labels-idx1-ubyte.gz  

### MNIST数据集转换

`MnistToMR`这个类用于将MNIST数据集转换为MindSpore数据格式，参数用法如下：
- `source` - 包含t10k-images-idx3-ubyte.gz，train-images-idx3-ubyte.gz，t10k-labels-idx1-ubyte.gz，train-labels-idx1-ubyte.gz的目录，本例中使用变量`mnist_path`传入该参数。
- `destination` - 要转换成MindSpore数据格式文件的目录，本例中使用变量`mnist_mindrecord_path`传入该参数。
- `partition_number` - 分区的大小，默认为1，本例中使用的默认参数。

In [2]:
from mindspore.mindrecord import MnistToMR

mnist_path = './dataset/MnistData'
mnist_mindrecord_path = './transform/mnist.record'

mnist_transformer = MnistToMR(mnist_path,mnist_mindrecord_path)
# executes transformation from Mnist to MindRecord
mnist_transformer.transform()

MSRStatus.SUCCESS

因为MNIST数据集包含训练数据集以及测试数据集，所以生成的MindSpore数据格式文件也分别是训练数据集和测试数据集的，其中.db结尾的文件保存的是描述MindSpore数据格式文件的元数据信息，切记一定不要删除它，生成的文件如下所示：

    transform/mnist.record_test.mindrecord   
    transform/mnist.record_test.mindrecord.db   
    transform/mnist.record_train.mindrecord   
    transform/mnist.record_train.mindrecord.db  

以下内容首先加载MindSpore数据格式的数据集，本例只进行了训练数据集的加载，如果需要测试数据集的加载也是同样的操作，然后对数据创建了字典迭代器，并通过迭代器读取了一条数据记录。

In [3]:
file_name = './transform/mnist.record_train.mindrecord'
# create MindDataset for reading data
mnist_data_set = ds.MindDataset(dataset_file=file_name)
# create a dictionary iterator and read a data record through the iterator
print(next(mnist_data_set.create_dict_iterator()))

{'data': array([255, 216, 255, 224,   0,  16,  74,  70,  73,  70,   0,   1,   1,
         0,   0,   1,   0,   1,   0,   0, 255, 219,   0,  67,   0,   2,
         1,   1,   1,   1,   1,   2,   1,   1,   1,   2,   2,   2,   2,
         2,   4,   3,   2,   2,   2,   2,   5,   4,   4,   3,   4,   6,
         5,   6,   6,   6,   5,   6,   6,   6,   7,   9,   8,   6,   7,
         9,   7,   6,   6,   8,  11,   8,   9,  10,  10,  10,  10,  10,
         6,   8,  11,  12,  11,  10,  12,   9,  10,  10,  10, 255, 192,
         0,  11,   8,   0,  28,   0,  28,   1,   1,  17,   0, 255, 196,
         0,  31,   0,   0,   1,   5,   1,   1,   1,   1,   1,   1,   0,
         0,   0,   0,   0,   0,   0,   0,   1,   2,   3,   4,   5,   6,
         7,   8,   9,  10,  11, 255, 196,   0, 181,  16,   0,   2,   1,
         3,   3,   2,   4,   3,   5,   5,   4,   4,   0,   0,   1, 125,
         1,   2,   3,   0,   4,  17,   5,  18,  33,  49,  65,   6,  19,
        81,  97,   7,  34, 113,  20,  50, 129, 145, 161

## 将CSV数据集转换为MindSpore数据格式

### CSV数据集下载

- 本例中需要的数据位置在https://gitee.com/mindspore/docs/tree/master/tutorials/notebook/convert_dataset_to_mindspore_data_format/csv_data/data.csv
中，使用过程中可以在此路径下找到文件并下载，并且保存在`jupyter工作目录/dataset/`下，如图所示：

    dataset/data.csv

### CSV数据集转换

`CsvToMR`这个类用于将CSV数据集转换为MindSpore数据格式，参数用法如下：
- `source` - CSV的文件的路径，本例中使用变量`csv_path`传入该参数。
- `destination` - 要转换成MindSpore数据格式文件的目录，本例使用变量`csv_mindrecord_path`传入该参数。
- `columns_list` - 要读取的列的列表，默认为None，本例使用默认参数。
- `partition_number` - 分区的大小，默认为1，本例使用默认参数。

In [4]:
from mindspore.mindrecord import CsvToMR
from mindspore.mindrecord import FileReader
csv_path = './dataset/data.csv'
csv_mindrecord_path = './transform/csv.mindrecord'

csv_transformer = CsvToMR(csv_path,csv_mindrecord_path) 
# executes transformation from Csv to MindRecord
csv_transformer.transform()

MSRStatus.SUCCESS

生成的文件如下所示：

    transform/csv.mindrecord   
    transform/csv.mindrecord.db   

以下内容首先加载MindSpore数据格式的数据集，然后对数据创建了字典迭代器，并通过迭代器读取了一条数据记录。

In [5]:
# create MindDataset for reading data
csv_data_set = ds.MindDataset(dataset_file=csv_mindrecord_path)
# create a dictionary iterator and read a data record through the iterator
print(next(csv_data_set.create_dict_iterator()))

{'b': array(b'q', dtype='|S1'), 'c': array(b'e', dtype='|S1'), 'a': array(b'w', dtype='|S1')}


## 将CIFAR-10数据集转换为MindSpore数据格式

### CIFAR-10数据集下载

- CIFAR-10数据集：
   > https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
- 将下载好的CIFAR-10数据集放在`jupyter工作目录/dataset/Cifar10Data/`下，如图所示：

    dataset/Cifar10Data/  
    ├── batches.meta  
    ├── data_batch_1  
    ├── data_batch_2  
    ├── data_batch_3  
    ├── data_batch_4  
    ├── data_batch_5  
    ├── readme.html  
    └── test_batch  

### CIFAR-10数据集转换

`Cifar10ToMR`这个类用于将CIFAR-10数据集转换为MindSpore数据格式，参数用法如下：
- `source` - 存放CIFAR-10数据集的目录，本例中使用变量`cifar10_path`传入该参数。
- `destination` - 要转换成MindSpore数据格式文件的目录，本例中使用变量`cifar10_mindrecord_path`传入该参数。

In [6]:
from mindspore.mindrecord import Cifar10ToMR

cifar10_path = './dataset/Cifar10Data/'
cifar10_mindrecord_path = './transform/cifar10.record'

cifar10_transformer = Cifar10ToMR(cifar10_path,cifar10_mindrecord_path)
# executes transformation from Cifar10 to MindRecord
cifar10_transformer.transform(['label'])

MSRStatus.SUCCESS

因为CIFAR-10数据集包含训练数据集以及测试数据集，所以生成的MindSpore数据格式文件也分别是训练数据集和测试数据集的，生成的文件如下所示：

    transform/cifar10.record  
    transform/cifar10.record.db  
    transform/cifar10.record_test   
    transform/cifar10.record_test.db   

以下内容首先加载MindSpore数据格式的数据集，本例只进行了训练数据集的加载，如果需要测试数据集的加载也是同样的操作，然后对数据创建了字典迭代器，并通过迭代器读取了一条数据记录。

In [7]:
# create MindDataset for reading data
cifar10_data_set = ds.MindDataset(dataset_file=cifar10_mindrecord_path)
# create a dictionary iterator and read a data record through the iterator
print(next(cifar10_data_set.create_dict_iterator()))

{'data': array([255, 216, 255, ...,  35, 255, 217], dtype=uint8), 'id': array(30707, dtype=int64), 'label': array(4, dtype=int64)}


## 将CIFAR-100数据集转换为MindSpore数据格式

### CIFAR-100数据集下载

- CIFAR-100数据集：
   > https://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz

- 将下载好的CIFAR-100数据集放在`jupyter工作目录/dataset/Cifar100Data/`下，如图所示：

    dataset/Cifar100Data/  
    ├── file.txt~  
    ├── meta  
    ├── test  
    └── train  

### CIFAR-100数据集转换

`Cifar100ToMR`这个类用于将CIFAR-100数据集转换为MindSpore数据格式，参数用法如下：
- `source` - 存放CIFAR-100数据集的目录，本例中使用变量`cifar100_path`传入该参数。
- `destination` - 要转换成MindSpore数据格式文件的目录，本例中使用变量`cifar100_mindrecord_path`传入该参数。

In [8]:
from mindspore.mindrecord import Cifar100ToMR

cifar100_path = './dataset/Cifar100Data/'
cifar100_mindrecord_path = './transform/cifar100.record'

cifar100_transformer = Cifar100ToMR(cifar100_path,cifar100_mindrecord_path)
#executes transformation from Cifar100 to MindRecord
cifar100_transformer.transform(['fine_label','coarse_label'])

MSRStatus.SUCCESS

因为CIFAR-100数据集包含训练数据集以及测试数据集，所以生成的MindSpore数据格式文件也分别是训练数据集和测试数据集的，生成的文件如下所示：

    transform/cifar100.record  
    transform/cifar100.record.db  
    transform/cifar100.record_test  
    transform/cifar100.record_test.db  

以下内容首先加载MindSpore数据格式的数据集，本例只进行了训练数据集的加载，如果需要测试数据集的加载也是同样的操作，然后对数据创建了字典迭代器，并通过迭代器读取了一条数据记录。

In [9]:
# create MindDataset for reading data
cifar100_data_set = ds.MindDataset(dataset_file=cifar100_mindrecord_path)
# create a dictionary iterator and read a data record through the iterator
print(next(cifar100_data_set.create_dict_iterator()))

{'data': array([255, 216, 255, ..., 127, 255, 217], dtype=uint8), 'fine_label': array(88, dtype=int64), 'coarse_label': array(8, dtype=int64), 'id': array(10349, dtype=int64)}


## 将ImageNet数据集转换为MindSpore数据格式

### ImageNet数据集下载

- ImageNet数据集：
   > http://image-net.org/download
- 将下载好的数据集放在`jupyter工作目录/dataset/ImageNetData/`下，如图所示：

    dataset/ImageNetData/    
    ├── bounding_boxes   
    ├── imagenet_map.txt     
    ├── train    
    └── validation    

### ImageNet数据集转换

`ImageNetToMR`这个类用于将ImageNet数据集转换为MindSpore数据格式，参数用法如下：
- `map_file` - map file 应该显示标签，内容如下所示：

            n02119789  0  
            n02100735  1  
            n02110185  2  
            n02096294  3  
本例中使用变量`imagenet_map_path`传入该参数。


- `image_dir` - image目录应该是包含n02119789、n02100735、n02110185、n02096294的目录，本例中使用变量`imagenet_image_dir`传入该参数。
- `destination` - 要转换成MindSpore数据格式文件的路径，本例中使用变量`imagenet_mindrecord_path`传入该参数。
- `partititon_number` - 分区的大小，本例中设置该值为4，表示会生成4个MindSpore格式文件，使用变量`partition_number`传入该参数。

In [10]:
from mindspore.mindrecord import ImageNetToMR

imagenet_map_path = './dataset/ImageNetData/imagenet_map.txt'
imagenet_image_dir = './dataset/ImageNetData/train'
imagenet_mindrecord_path = './transform/imagenet.record'
partition_number = 4

imagenet_transformer = ImageNetToMR(imagenet_map_path,imagenet_image_dir,imagenet_mindrecord_path,partition_number)
#executes transformation from ImageNet to MindRecord
imagenet_transformer.transform()

MSRStatus.SUCCESS

因为本例中指定了分区大小为4，所以会生成4个MindSpore数据格式文件，因为数据集比较大，转换时间有点长，请耐心等待，生成的文件如下所示：

    transform/imagenet.record0  
    transform/imagenet.record0.db  
    transform/imagenet.record1  
    transform/imagenet.record1.db   
    transform/imagenet.record2   
    transform/imagenet.record2.db  
    transform/imagenet.record3  
    transform/imagenet.record3.db  

以下内容首先加载MindSpore数据格式的数据集，虽然是自己指定了分区，但是文件之间还是相互有联系，所以不管对任何一个MindSpore数据格式文件加载，都会同时把其他三个文件一起加载，本例将imagenet.record0文件进行了加载，然后对数据创建了字典迭代器，并通过迭代器读取了一条数据记录。

In [11]:
file_name = './transform/imagenet.record0'
# create MindDataset for reading data
imagenet_data_set = ds.MindDataset(dataset_file=file_name)
# create a dictionary iterator and read a data record through the iterator
print(next(imagenet_data_set.create_dict_iterator()))

{'label': array(564, dtype=int64), 'file_name': array(b'./dataset/ImageNetData/train/n03388183/n03388183_8017.JPEG',
      dtype='|S58'), 'data': array([255, 216, 255, ...,  39, 255, 217], dtype=uint8)}


## 用户自定义生成MindSpore数据格式

1. 导入`FileWriter`类，用于将用户定义的原始数据写入，参数用法如下：


- `file_name` - MindSpore数据格式文件的文件名，本例使用变量`data_record_path`传入该参数。
- `shard_num` - MindSpore数据格式文件的数量，默认为1，取值范围在[1,1000]，本例使用默认参数。

In [12]:
from mindspore.mindrecord import FileWriter
data_record_path = './transform/data.record'
writer = FileWriter(data_record_path,1)

2. 定义数据集Schema，Schema用于定义数据集包含哪些字段以及字段的类型，然后添加Schema，相关规范如下：


- 字段名：字母、数字、下划线。
- 字段属性type：int32、int64、float32、float64、string、bytes。
- 字段属性shape：如果是一维数组，用[-1]表示，如果是二维数组，用[m,n]表示，如果是三维数组，用[x,y,z]表示。

本例中定义了`file_name`字段，用于标注准备写入数据的文件名字，定义了`label`字段，用于给数据打标签，定义了`data`字段，用于保存数据。

In [13]:
data_schema = {"file_name":{"type":"string"},"label":{"type":"int32"},"data":{"type":"bytes"}}
writer.add_schema(data_schema,"test_schema")

0

3. 准备需要写入的数据，按照用户定义的Schema形式，准备需要写入的样本列表，本例中需要的数据位置在https://gitee.com/mindspore/docs/tree/master/tutorials/notebook/convert_dataset_to_mindspore_data_format/images/transform.jpg
中，使用过程中可以在此路径下找到图片并下载，并且保存在`jupyter工作目录/dataset/`下。

In [14]:
def image_to_bytes(file_name):
    f = open(file_name,'rb')
    image_bytes = f.read()
    f.close()
    return image_bytes

data = [{"file_name":"transform.jpg","label":1,"data":image_to_bytes('./dataset/transform.jpg')}]

4. 添加索引字段，添加索引字段可以加速数据读取，改步骤为可选操作。

In [15]:
indexes = ["file_name","label"]
writer.add_index(indexes)

MSRStatus.SUCCESS

5. 写入数据，最后生成MindSpore数据格式文件，`write_raw_data`接口可以被重复调用，方便用户将多个样本添加至MindSpore数据格式文件中。

In [16]:
writer.write_raw_data(data)
writer.commit()

MSRStatus.SUCCESS

6. 以下内容首先加载MindSpore数据格式的数据集，然后对数据创建了字典迭代器，并通过迭代器读取了一条数据记录。

In [18]:
file_name = './transform/data.record'
# create MindDataset for reading data
define_data_set = ds.MindDataset(dataset_file=file_name)
# create a dictionary iterator and read a data record through the iterator
print(next(define_data_set.create_dict_iterator()))

{'label': array(1, dtype=int32), 'file_name': array(b'transform.jpg', dtype='|S13'), 'data': array([255, 216, 255, ..., 159, 255, 217], dtype=uint8)}


## 总结

以上便是本次体验的内容，我们通过此次体验全面了解了如何通过各个相应的子模块将其他格式的数据集转换为MindSpore数据格式。